# Signature Matrix Generation

This notebook documents steps used to generate a siganture matrix of scRNA-seq data for CIBERSORTx.

Revelant links:
1. The CIBERSORTx paper describing the workflow for deconvolution that uses the signature matrix: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7695353/

2. The publication with reference snRNAseq  data for nucleus accumbens: https://www.sciencedirect.com/science/article/pii/S0896627321006553

3. The link to their deposited reference data: https://github.com/LieberInstitute/10xPilot_snRNAseq-human#readme

4. Source code for Seurat function that builds a signature matrix: https://rdrr.io/github/PelzKo/dwls/man/buildSignatureMatrixUsingSeruat.html

5. An alternative R package for building signature matrices: https://github.com/ruppinlab/scSigR


## 1. Download data

The data downloaded are [processed data](https://github.com/LieberInstitute/10xPilot_snRNAseq-human?tab=readme-ov-file#processed-data). They are in the format of corresponding SingleCellExperiment R/Bioconductor objects (with reducedDims, annotations, etc.) for each of the five regions across eight donors:

-  AMY
- DLPFC
- HPC
- NAc
- sACC

In [1]:
cd ~/project/git/sigmat/data/
wget https://libd-snrnaseq-pilot.s3.us-east-2.amazonaws.com/SCE_NAc-n8_tran-etal.rda

## 2. Data inspection

In [2]:
setwd("~/project/git/sigmat/")

library(SingleCellExperiment)
library(dplyr)
library(Seurat)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [3]:
load("./data/SCE_NAc-n8_tran-etal.rda")
ls()

[1] "PATH"         "sce.nac.tran"

In [5]:
## barcode for cells
colData(sce.nac.tran) %>% dim()
colData(sce.nac.tran) %>% colnames()
colData(sce.nac.tran) %>% rownames() %>% head()

[1] 20571    14

[1] "Barcode"          "sum"              "detected"         "doubletScore"    
 [5] "region"           "donor"            "sex"              "processBatch"    
 [9] "protocol"         "sequencer"        "sizeFactor"       "prelimCluster"   
[13] "collapsedCluster" "cellType"

[1] "donor1_AAACCCACATCGAACT.1" "donor1_AAACCCATCCAACCAA.1"
[3] "donor1_AAACGAACAATGACCT.1" "donor1_AAACGAATCACGAGGA.1"
[5] "donor1_AAACGCTAGGTTGAGC.1" "donor1_AAACGCTCACTGCATA.1"

In [6]:
## gene names
rowData(sce.nac.tran) %>% dim()
rowData(sce.nac.tran) %>% colnames()
rowData(sce.nac.tran) %>% rownames() %>% head()

[1] 33538    36

[1] "gene_id"                    "gene_version"              
 [3] "gene_name"                  "gene_source"               
 [5] "gene_biotype"               "Symbol.uniq"               
 [7] "propNucleiExprs"            "propExprsIn.Astro_A"       
 [9] "propExprsIn.Astro_B"        "propExprsIn.drop.doublet_A"
[11] "propExprsIn.drop.doublet_B" "propExprsIn.drop.doublet_C"
[13] "propExprsIn.drop.doublet_D" "propExprsIn.drop.lowNTx"   
[15] "propExprsIn.Inhib_A"        "propExprsIn.Inhib_B"       
[17] "propExprsIn.Inhib_C"        "propExprsIn.Inhib_D"       
[19] "propExprsIn.Inhib_E"        "propExprsIn.Macrophage"    
[21] "propExprsIn.Micro"          "propExprsIn.Micro_resting" 
[23] "propExprsIn.MSN.D1_A"       "propExprsIn.MSN.D1_B"      
[25] "propExprsIn.MSN.D1_C"       "propExprsIn.MSN.D1_D"      
[27] "propExprsIn.MSN.D1_E"       "propExprsIn.MSN.D1_F"      
[29] "propExprsIn.MSN.D2_A"       "propExprsIn.MSN.D2_B"      
[31] "propExprsIn.MSN.D2_C"       "propExprsIn.MSN.D2_D"      
[33] "propExprsIn.Oligo_A"        "propExprsIn.Oligo_B"       
[35] "propExprsIn.OPC"            "propExprsIn.OPC_COP"

[1] "MIR1302-2HG" "FAM138A"     "OR4F5"       "AL627309.1"  "AL627309.3" 
[6] "AL627309.2"

In [25]:
# single cell expression matrix
assayNames(sce.nac.tran)

[1] "counts"    "logcounts"

In [35]:
assay(sce.nac.tran, "logcounts") %>% head()

  [[ suppressing 34 column names ‘donor1_AAACCCACATCGAACT.1’, ‘donor1_AAACCCATCCAACCAA.1’, ‘donor1_AAACGAACAATGACCT.1’ ... ]]



6 x 20571 sparse Matrix of class "dgCMatrix"
                                                                               
MIR1302-2HG . . . . . . . . . . . . . . . . . . . . .         . .         . . .
FAM138A     . . . . . . . . . . . . . . . . . . . . .         . .         . . .
OR4F5       . . . . . . . . . . . . . . . . . . . . .         . .         . . .
AL627309.1  . . . . . . . . . . . . . . . . . . . . 0.1875968 . 0.6843966 . . .
AL627309.3  . . . . . . . . . . . . . . . . . . . . .         . .         . . .
AL627309.2  . . . . . . . . . . . . . . . . . . . . .         . .         . . .
                                          
MIR1302-2HG . . .         . . . . . ......
FAM138A     . . .         . . . . . ......
OR4F5       . . .         . . . . . ......
AL627309.1  . . 0.9035047 . . . . . ......
AL627309.3  . . .         . . . . . ......
AL627309.2  . . .         . . . . . ......

 .....suppressing 20537 columns in show(); maybe adjust options(max.print=, width=)
 ........

## 3. Sigature Matrix Generation for NAc region

The code is adapted from: https://github.com/PelzKo/dwls/blob/master/R/functions.R based on `buildSignatureMatrixUsingSeurat` and `DEAnalysis` functions.

In [63]:
## Performing DE analysis using Seurat
##
## scdata: The single cell data matrix
## cell_types: A vector of the cell type annotations
##
## return: List with the differentially expressed genes for each cell type

DEAnalysis <- function(scdata, cell_types) {
    list.DE.group <- as.list(rep(0, length(cell_types)))
    for (c in cell_types) {
        de_group <- Seurat::FindMarkers(object = scdata,
                                        ident.1 = c,
                                        ident.2 = NULL,
                                        only.pos = TRUE,
                                        test.use = "bimod")

        index <- which(cell_types == c)
        list.DE.group[[index]] <- de_group
    }
    
    return(list.DE.group)
}

In [64]:
## Building the signature matrix using Seurat
##
## scdata: The single cell data matrix
## cell_types: A vector of the cell type annotations
## diff.cutoff: The FC cutoff
## pval.cutoff: The p-value cutoff
##
## return: Computed signature matrix

buildSignatureMatrixUsingSeurat <- function(scdata,
                                            cell_types,
                                            diff.cutoff = 0.5,
                                            pval.cutoff = 0.01){
    # perform differential expression analysis
    list.DE.groups <- DEAnalysis(scdata, cell_types)
    
    num_genes <- c()
    for (c in cell_types) {
        index <- which(cell_types == c)
        de_group <- list.DE.group[[index]]

        DEGenes <- rownames(de_group)[intersect(
            which(de_group$p_val_adj < pval.cutoff),
            which(de_group$avg_log2FC > diff.cutoff)
          )]

        ## why removing those genes ?
        nonMir <- grep("MIR|Mir", DEGenes, invert = TRUE)
        assign(
          paste("cluster_lrTest.table.", c, sep = ""),
          de_group[which(rownames(de_group) %in% DEGenes[nonMir]), ]
        )
        num_genes <- c(num_genes, length(DEGenes[nonMir]))
    }
    
    conditionNumbers <- c()
    for (g in 50:200) {
        Genes <- c()
        j <- 1
        for (i in cell_types) {
          if (num_genes[j] > 0) {
            temp <- paste("cluster_lrTest.table.", i, sep = "")
            temp <- get(temp)
            temp <- temp[order(temp$p_val_adj, decreasing = TRUE), ]
            Genes <-
              c(Genes, (rownames(temp)[1:min(g, num_genes[j])]))
          }
          j <- j + 1
        }

        ## gene names with ENSG are formatted differently 
        ## when converting SCE object to Seurat object
        Genes <- unique(Genes)
        Genes <- gsub("-ENSG", "_ENSG", Genes)

        ## make signature matrix
        ExprSubset <- scdata_nac_mat[Genes, , drop = FALSE]
        Sig <- NULL
        for (i in as.list(cell_types)) {
          Sig <-
            cbind(Sig, (apply(ExprSubset, 1, function(y) {
              mean(y[names(y) == i])
            })))
        }
        colnames(Sig) <- cell_types
        conditionNumbers <- c(conditionNumbers, kappa(Sig))
    }
    
    # g is optimal gene number
    g <- which.min(conditionNumbers) + min(49, num_genes - 1)
    Genes <- c()
    j <- 1
    for (i in cell_types) {
        if (num_genes[j] > 0) {
          temp <- paste("cluster_lrTest.table.", i, sep = "")
          temp <- get(temp)
          temp <- temp[order(temp$p_val_adj, decreasing = TRUE), ]
          Genes <-
            c(Genes, (rownames(temp)[1:min(g, num_genes[j])]))
        }
        j <- j + 1
    }

    Genes <- unique(Genes)
    Genes <- gsub("-ENSG", "_ENSG", Genes)
    ExprSubset <- scdata_nac_mat[Genes, , drop = FALSE]
    Sig <- NULL
    for (i in as.list(cell_types)) {
      Sig <-
        cbind(Sig, (apply(ExprSubset, 1, function(y) {
          mean(y[names(y) == i])
        })))
    }

    colnames(Sig) <- cell_types
    rownames(Sig) <- Genes
    
    return(Sig)
}

In [65]:
# convert SCE to Seurat object
scdata_nac <- as.Seurat(sce.nac.tran)

# set cell types
Idents(scdata_nac) <- scdata_nac@meta.data$cellType
cell_types <- scdata_nac@meta.data$cellType %>% unique() %>% as.character()

# retrieve expression matrix
scdata_nac_mat <- GetAssayData(scdata_nac, assay = "originalexp", slot = "data")
colnames(scdata_nac_mat) <- scdata_nac@meta.data$cellType

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from PCA_corrected_ to PCAcorrected_”
Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from PCA_opt_ to PCAopt_”


In [70]:
sigmat <- buildSignatureMatrixUsingSeurat(scdata_nac, cell_types)